In [1]:
import sys, string
import os
import socket
import time
from time import gmtime, strftime
import operator
import boto3
import json
import findspark
from pyspark.sql import SparkSession
from datetime import datetime


In [2]:
findspark.init()

In [3]:
spark = SparkSession\
        .builder\
        .appName("ethereum-sneha-C")\
        .getOrCreate()

In [4]:
def filter_transaction_line(line):
    try:
        fields = line.split(',')
        if len(fields) != 7 or int(fields[3]) ==0 or fields[0] == 'block_number':
            return False
        else:
            str(fields[2]) # convert to_address to string 
            return True
    except:
        return False

def filter_contracts_line(line):
    try:
        fields = line.split(',')
        if len(fields) != 5 or fields[0] == 'address':
            return False
        else:
            str(fields[0]) # convert address to string 
            return True
    except:
        return False

      
    


In [5]:
transactions_file = spark.sparkContext.textFile("./transactions.csv").filter(filter_transaction_line)
contracts_file = spark.sparkContext.textFile("./contract.csv").filter(filter_contracts_line)

In [7]:
weekly_gas_transactions = transactions_file.map(lambda x: (( x.split(',')[2], strftime('%Y-%m W%W', gmtime(int(x.split(',')[-1])))) , ( int(x.split(',')[4]), 1)))

weekly_gas_transactions_reduced = weekly_gas_transactions.reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))

weekly_gas_transactions_key_map = weekly_gas_transactions_reduced.map(lambda x: ( x[0][0], (x[0][1], x[1][0], x[1][1])))

weekly_gas_transactions_contracts_joined = weekly_gas_transactions_key_map.leftOuterJoin(contracts_file.map(lambda x: ( x.split(',')[0], 'contract')))

weekly_gas_transactions_contracts_key_map = weekly_gas_transactions_contracts_joined.map(lambda x: (('Wallet' if x[1][1] is None else x[1][1], x[1][0][0]),(x[1][0][1], x[1][0][2])))

weekly_gas_transactions_contracts_reduced = weekly_gas_transactions_contracts_key_map.reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))
step7 = weekly_gas_transactions_contracts_reduced.mapValues(lambda x: x[0]/x[1])
step8 = step7.map(lambda x: (x[0][1], (x[0][0], x[1])))
step9 = step8.sortByKey()

In [8]:
step9.collect()[:5]

[('2016-09 W37', ('Wallet', 62250.0)), ('2016-09 W37', ('contract', 55500.0))]